[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ltdaovn/dataset/blob/master/TheGioiDiDong/thegioididong_crawling_28_08.ipynb)

# install

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install html5lib
!pip install lxml

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 4.8 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,294 kB]
Hit:12 http://ppa.launchpad.ne

# import

In [6]:
import sys
from selenium import webdriver
import time

import bs4
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json

# setting

In [7]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [8]:
SESSION = requests.Session()
HEADERS = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/48.0.2564.109 Safari/537.36')
}

BASE_URL = 'https://www.thegioididong.com/'

# function

In [9]:
def crawling(url,time_sleeping = 10):
    print('- webdriver is starting ... ')
    wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    wd.get(url)
    time.sleep(time_sleeping)
    page = wd.page_source
    soup = BeautifulSoup(page, 'html.parser')
    wd.close()

    return soup

In [10]:
# lấy link những item xuất hiện trên một trang đã được tải đầy đủ theo type và end_page
def get_all_item_in_page(type,end_page):

    url = BASE_URL + type + str(end_page)
    print(url)
    soup = crawling(url)

    all_item = soup.find_all('li', {'class': ['item','ajaxed',' item ']})

    if len(all_item) > 0:
        print('- Found {number:} items'.format(number = len(all_item)))
        # href_all_item_in_page = [a.find('a').get('href') for a in all_item]
        href_all_item_in_page = []

        for a in all_item:
            try:
                h = a.find('a').get('href')
                href_all_item_in_page.append(h)
            except:
                print(a)

        if len(href_all_item_in_page) > 0:
            print('- Found links of {number:} items'.format(number = len(href_all_item_in_page)))
            return href_all_item_in_page
        else:
            print('- Not find the link of the item')
            return None
    else:
        print('- No item found')
        return None

In [11]:
#get_all_item_in_page('dtdd#c=42&o=9&pi=',10)

In [12]:
def rating_processing(rating):
    num_star = 0
    comment_star = rating.find('div',{'class':['comment-star']})
    for star in comment_star.find_all('i'):
        if star['class'][0] == 'icon-star':
            num_star += 1

    cmt_txt = rating.find('p',{'class':['cmt-txt']})
    comment = cmt_txt.text

    txtname = rating.find('p',{'class':['txtname']})
    username = txtname.text

    rating = {
        'username': username,
        'comment': comment,
        'star': num_star,
        }
    print(rating)
    return rating

In [13]:
def get_rating(rating_page_url,session, headers, except_loop = 8,num_page = 1):

    try:
        url = rating_page_url+'?p='+str(num_page)
        response = session.get(url,headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        except_loop -= 1
        if except_loop > 0:
            session = requests.Session() #requests lại
            get_rating(rating_page_url, session, headers ,except_loop = except_loop,num_page = num_page)
        else:
            raise
    rating_data = []
    rating_list = soup.find_all('div',{'class':['comment__item par']})
    if len(rating_list) > 0:
        for rating in rating_list:
            rating_data.append(rating_processing(rating))

    return rating_data

rating_page_url = 'https://www.thegioididong.com/dtdd/realme-c12/danh-gia'
rating_list = get_rating(rating_page_url , SESSION, HEADERS, except_loop = 8,num_page = 1)
len(rating_list)

{'username': 'Lê Quỳnh Trang', 'comment': 'Nút âm lượng chỉnh không được mà ứng dụng bị lỗi máy bị trầy mặt lưng 1 chút do mình làm rơi có được bảo hành ko ạ', 'star': 3}
{'username': 'Minh Quan', 'comment': 'Anh ơi cho em hỏi realme c12 mới được hệ thống cập nhật mới hôm qua mà sau khi cập nhật xong điện thoại có hiện tượng giật lag chơi game ,lướt facebook , nhắn tin ,... Bị giật lag liên tục em đã khởi động lại điện thoại mà nó vẫn như vậy:((', 'star': 2}
{'username': 'Lương Hồng nhung', 'comment': 'Máy có hình ảnh giao diện mờ ảo kém.ko chia sẻ đc mạng 4g.thất vọng khi mua mưa đc hơn 1 tháng giảm gần 1trieu.chán thật sư\n', 'star': 2}
{'username': 'Lê văn khánh', 'comment': 'Máy chậm hơn máy cũ mua 2tr tôi đang dùng.máy chỉ vào đọc báo và fb mà thường xuyên bị đơ, đặc biệt lúc nhắn tin zalo hay truy cập tài khoản trên web.', 'star': 2}
{'username': 'Nguyễn Văn Tân', 'comment': 'Dùng được 3 ngày cảm thấy rất hài lòng với sp hơn 2tr thì k máy nào tốt hơn được máy này', 'star': 4}
{'u

30

In [14]:
def crawling_rating_data():
    print("main runing ...")

    categories = [('dtdd#c=42&o=9&pi=',14),('laptop#c=44&o=9&pi=',8),('may-tinh-bang#c=522&o=9&pi=',2)]

    link_all_items = []
    for type, end_page in categories:
        link_all_items[len(link_all_items):] = get_all_item_in_page(type,end_page)

    print('- Found {number:} items for all'.format(number = len(link_all_items)))


    data = []
    for link_item in link_all_items:
        if '?src=osp' in link_item:
            link_item = link_item.replace('?src=osp', "")

        rating_page_url = 'https://www.thegioididong.com'+link_item+'/danh-gia'
        print(rating_page_url)

        t = True
        n = 1
        while t:
            rating_data = get_rating(rating_page_url,SESSION, HEADERS, except_loop = 8, num_page = n)
            if rating_data == []:
                t = False
            else:
                data.append(rating_data)
                n += 1

    
    return data

# crawling

In [15]:
rating_data = crawling_rating_data()
# thegioididong_rating_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/kkdl/thegioididong_rating_data_12_06_21.json'
# with open(thegioididong_rating_data_path, 'w', encoding='utf-8') as file: # file nằm ở thư mục cá nhân
#     json.dump(rating_data, file,ensure_ascii=False)

main runing ...
https://www.thegioididong.com/dtdd#c=42&o=9&pi=14
- webdriver is starting ... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


- Found 181 items
<li class="merge__item item act" data-id="213031" data-index="0" data-url="/dtdd/iphone-12">64GB</li>
<li class="merge__item item " data-id="228736" data-index="1" data-url="/dtdd/iphone-12-128gb">128GB</li>
<li class="merge__item item " data-id="228737" data-index="2" data-url="/dtdd/iphone-12-256gb">256GB</li>
<li class="merge__item item act" data-id="213033" data-index="0" data-url="/dtdd/iphone-12-pro-max">128GB</li>
<li class="merge__item item " data-id="228743" data-index="1" data-url="/dtdd/iphone-12-pro-max-256gb">256GB</li>
<li class="merge__item item " data-id="228744" data-index="2" data-url="/dtdd/iphone-12-pro-max-512gb">512GB</li>
<li class="merge__item item act" data-id="213032" data-index="0" data-url="/dtdd/iphone-12-pro">128GB</li>
<li class="merge__item item " data-id="228738" data-index="1" data-url="/dtdd/iphone-12-pro-256gb">256GB</li>
<li class="merge__item item " data-id="228739" data-index="2" data-url="/dtdd/iphone-12-pro-512gb">512GB</li>
<l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


- Found 148 items
- Found links of 148 items
https://www.thegioididong.com/may-tinh-bang#c=522&o=9&pi=2
- webdriver is starting ... 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


Streaming output truncated to the last 5000 lines.
{'username': 'Trịnh Anh Tuấn', 'comment': 'Mik mới mua vsmart joy 4 3gb/64 (ngày 12-1-2021) mà h mik muốn đổi máy này lấy bản 6gb/64 và bù thêm có được không ', 'star': 4}
{'username': 'Trần văn', 'comment': 'Em hỏi cách bật đèn led thông báo sạc phin ạ. Tìm mãi không thấy có đèn led thông báo', 'star': 4}
{'username': 'le dung sy', 'comment': ' camera thì chụp thua dt cũ huawe y9 2019 . trong khi joy thi camera 16 còn huawe thì 13   ứng dung vào hay thoát ra ra đều ra đều thấy chậm.. nhiều khi tự cho cho minh hơn. ứng dụng dụng thời tiết. thì thì tự thì tự đổi thành vsmart thông minh. mới ghê.  trong trong máy mình vào phần phần giới thiệu về dt. chả thấy gi. là ram bo nhiêu còn bao nhiêu. rồi bộ nhớ. chả có. nói chung ông vuong doi lúc chỉ hiểu về bất động sản. thôi. chứ về các lĩnh vực khác. chưa chắc đã hiểu.. mình mua con này trả góp nên khong đổi dc. qua hãng khác .   nói chung đây là suy nghĩ cua mình. . ', 'star': 2}
{'username

In [18]:
from datetime import datetime
#print(datetime.today().strftime('%Y%m%d'))
thegioididong_rating_data_path = '/content/drive/MyDrive/Colab Notebooks/thegioididong_rating_data_' + datetime.today().strftime('%Y%m%d') + '.json'
with open(thegioididong_rating_data_path, 'w', encoding='utf-8') as file: # file nằm ở thư mục cá nhân
    json.dump(rating_data, file,ensure_ascii=False)